In [2]:
from sklearn.model_selection import KFold, train_test_split
import torch.distributions.distribution as D
import data.medain_filtering_class as mf
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torch.optim as optim
from RBM import RBM
import numpy as np
import datetime
import torch

print(datetime.datetime.now(), "model.py code start")

BATCH_SIZE = 10
EPOCH = 100
LEARNING_RATE = 0.2
ANNEALING_RATE = 0.999
VISIBLE_UNITS = [180, 200, 250]
HIDDEN_UNITS = [80, 100, 120]
K_FOLD = 1

2022-05-04 10:47:33.997652 model.py code start


In [3]:
print("[MODL] Model main code is starting....")

print("[INFO] Read train data, cross-vaildation data and test data from median filtering code")
dataset_db1, dataset_db2, dataset_db3 = mf.ecg_filtering(True)

train_dataset = list(mf.list_to_list(dataset_db1)) * 4
cross_dataset = list(mf.list_to_list(dataset_db2)) * 4
test_dataset = list(mf.list_to_list(dataset_db3))  * 4

X_train, X_test, y_train, y_test = train_test_split(
    (train_dataset + cross_dataset), 
    (test_dataset + cross_dataset),
    test_size=0.33,
    shuffle=True
)

[MODL] Model main code is starting....
[INFO] Read train data, cross-vaildation data and test data from median filtering code
[INFO] Read file and indexing start...
[INFO]	final_db1 direcotry found.
......	...................i	Current_Index	From_Index
[IWIP]	final_db1 reading... 0 0 200
[IWIP]	final_db1 reading... 1 200 400
[IWIP]	final_db1 reading... 2 400 600
[IWIP]	final_db1 reading... 3 600 800
[IWIP]	final_db1 reading... 4 800 1000
[IWIP]	final_db1 reading... 5 1000 1200
[IWIP]	final_db1 reading... 6 1200 1400
[IWIP]	final_db1 reading... 7 1400 1600
[IWIP]	final_db1 reading... 8 1600 1800
[IWIP]	final_db1 reading... 9 1800 2000
[IWIP]	final_db1 reading... 10 2000 2200
[IWIP]	final_db1 reading... 11 2200 2400
[IWIP]	final_db1 reading... 12 2400 2600
[IWIP]	final_db1 reading... 13 2600 2800
[IWIP]	final_db1 reading... 14 2800 3000
[IWIP]	final_db1 reading... 15 3000 3200
[IWIP]	final_db1 reading... 16 3200 3400
[IWIP]	final_db1 reading... 17 3400 3600
[IWIP]	final_db1 reading... 18 

/Users/bahk_insung/miniforge3/lib/python3.9/site-packages/scipy/signal/_signaltools.py:1531: UserWarning: kernel_size exceeds volume extent: the volume will be zero-padded.
  warnings.warn('kernel_size exceeds volume extent: the volume will be '


[DONE] Pre-processing is done.
[ERRR] Got a issue with create file


In [4]:
train_dataloader = DataLoader(X_train + y_train,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

test_dataloader = DataLoader(X_test + y_test,
                             batch_size=BATCH_SIZE,
                             shuffle=True)

print("X_train length : ", len(X_train))
print("X_test  length : ", len(X_test))
print("y_train length : ", len(y_train))
print("y_test  length : ", len(y_test))

train_data = torch.FloatTensor(X_train)
test_data = torch.FloatTensor(X_test)

X_train length :  9648
X_test  length :  4752
y_train length :  9648
y_test  length :  4752


In [5]:
print("[INFO] Model object added")

rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE)

first_train_op = optim.SGD(rbm_first.parameters(), 0.1)
second_train_op = optim.SGD(rbm_second.parameters(), 0.1)
third_train_op = optim.SGD(rbm_third.parameters(), 0.1)

gb_first_train_op = optim.SGD(rbm_first.parameters(), 0.1)
gb_second_train_op = optim.SGD(rbm_second.parameters(), 0.1)
gb_third_train_op = optim.SGD(rbm_third.parameters(), 0.1)

output_from_first = list()
output_from_second = list()
output_from_third = list()

[INFO] Model object added


In [6]:
'''Train Part'''

loss_ = []
for epoch in range(EPOCH):
    '''First bbrbm'''
    for _, (data) in enumerate(train_dataloader):
        try:
            # tnesor float
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue

        sample_data = torch.bernoulli(data)
        sample_data = torch.flatten(sample_data.clone())

        # tensor binary
        vog_first, v1, mt = rbm_first(sample_data)
        
        loss_first = rbm_first.free_energy(vog_first) - rbm_first.free_energy(v1)
        loss_.append(loss_first.data)
        
        first_train_op.zero_grad()
        loss_first.backward()
        first_train_op.step()
    
    output_from_first.append(v1.tolist())
    print("1ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, np.mean(loss_), mt))

output_from_first = torch.tensor(output_from_first)
for epoch in range(EPOCH):
    '''Secnd bbrbm'''
    for _, (data) in enumerate(output_from_first):
        try:
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue

        sample_data = torch.bernoulli(data)
        sample_data = torch.flatten(sample_data.clone())

        vog_second, v2, mt = rbm_second(sample_data)
        
        loss_second = rbm_second.free_energy(vog_second) - rbm_second.free_energy(v2)
        loss_.append(loss_second.data)
        
        second_train_op.zero_grad()
        loss_second.backward()
        second_train_op.step()

    output_from_second.append(v2.tolist())
    print("2ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : ".format(epoch, np.mean(loss_), mt))

output_from_second = torch.tensor(output_from_second)
for epoch in range(EPOCH):
    '''Third bbrbm'''
    for _, (data) in enumerate(output_from_second):
        try:
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue

        sample_data = torch.bernoulli(data)
        sample_data = torch.flatten(sample_data.clone())

        vog_third, v3, mt = rbm_third(sample_data)
        
        loss_third = rbm_third.free_energy(vog_third) - rbm_third.free_energy(v3)
        loss_.append(loss_third.data)
        
        third_train_op.zero_grad()
        loss_third.backward()
        third_train_op.step()

    output_from_third.append(v3.tolist())
    print("3ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : ".format(epoch, np.mean(loss_), mt))

/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_38366/3670610706.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
/Users/bahk_insung/miniforge3/lib/python3.9/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


1ST BBrbm_first Training loss for 0 epoch -0.137984037399292	Estimate time : 0:00:00.000105
1ST BBrbm_first Training loss for 1 epoch -0.040723636746406555	Estimate time : 0:00:00.000104
1ST BBrbm_first Training loss for 2 epoch -0.005773082375526428	Estimate time : 0:00:00.000104
1ST BBrbm_first Training loss for 3 epoch -0.037924546748399734	Estimate time : 0:00:00.000105
1ST BBrbm_first Training loss for 4 epoch -0.026829687878489494	Estimate time : 0:00:00.000107
1ST BBrbm_first Training loss for 5 epoch -0.08220463991165161	Estimate time : 0:00:00.000110
1ST BBrbm_first Training loss for 6 epoch 0.006887934636324644	Estimate time : 0:00:00.000104
1ST BBrbm_first Training loss for 7 epoch 0.006803278345614672	Estimate time : 0:00:00.000105
1ST BBrbm_first Training loss for 8 epoch 0.018773870542645454	Estimate time : 0:00:00.000107
1ST BBrbm_first Training loss for 9 epoch -0.006662246771156788	Estimate time : 0:00:00.000104
1ST BBrbm_first Training loss for 10 epoch 0.017865205183

/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_38366/3670610706.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)


2ST BBrbm_first Training loss for 0 epoch -0.0557221919298172	Estimate time : 
2ST BBrbm_first Training loss for 1 epoch -0.05571147799491882	Estimate time : 
2ST BBrbm_first Training loss for 2 epoch -0.0558512881398201	Estimate time : 
2ST BBrbm_first Training loss for 3 epoch -0.05602900683879852	Estimate time : 
2ST BBrbm_first Training loss for 4 epoch -0.055872008204460144	Estimate time : 
2ST BBrbm_first Training loss for 5 epoch -0.05593863129615784	Estimate time : 
2ST BBrbm_first Training loss for 6 epoch -0.05564093589782715	Estimate time : 
2ST BBrbm_first Training loss for 7 epoch -0.056047070771455765	Estimate time : 
2ST BBrbm_first Training loss for 8 epoch -0.056008342653512955	Estimate time : 
2ST BBrbm_first Training loss for 9 epoch -0.05587403103709221	Estimate time : 
2ST BBrbm_first Training loss for 10 epoch -0.05598278343677521	Estimate time : 
2ST BBrbm_first Training loss for 11 epoch -0.05598403513431549	Estimate time : 
2ST BBrbm_first Training loss for 12 

/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_38366/3670610706.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)


3ST BBrbm_first Training loss for 0 epoch -0.05619487166404724	Estimate time : 
3ST BBrbm_first Training loss for 1 epoch -0.05641287565231323	Estimate time : 
3ST BBrbm_first Training loss for 2 epoch -0.056574273854494095	Estimate time : 
3ST BBrbm_first Training loss for 3 epoch -0.056679777801036835	Estimate time : 
3ST BBrbm_first Training loss for 4 epoch -0.05632535368204117	Estimate time : 
3ST BBrbm_first Training loss for 5 epoch -0.056343600153923035	Estimate time : 
3ST BBrbm_first Training loss for 6 epoch -0.05626014992594719	Estimate time : 
3ST BBrbm_first Training loss for 7 epoch -0.056237366050481796	Estimate time : 
3ST BBrbm_first Training loss for 8 epoch -0.056522954255342484	Estimate time : 
3ST BBrbm_first Training loss for 9 epoch -0.05644404888153076	Estimate time : 
3ST BBrbm_first Training loss for 10 epoch -0.0564081184566021	Estimate time : 
3ST BBrbm_first Training loss for 11 epoch -0.056201085448265076	Estimate time : 
3ST BBrbm_first Training loss for

In [7]:
    
print("BBRBM is done.")
print("GBRBM is start")

output_from_first = list()
output_from_second = list()
output_from_third = torch.tensor(output_from_third)

rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE)

# print(output_from_third.size(), output_from_third.dim(), "\n", output_from_third)
gaussian_std = torch.arange(1, 0, -0.1)

for epoch in range(EPOCH):
    '''First gbrbm'''
    for _, (data) in enumerate(output_from_third):
        try:
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue
        
        # CHANGED to GAUSSIAN
        sample_data = torch.normal(mean=data, std=gaussian_std)
        sample_data = torch.flatten(sample_data.clone())

        gb_vog_first, gb_v1, mt = rbm_first(sample_data)
        
        gb_loss_first = rbm_first.free_energy(gb_vog_first) - rbm_first.free_energy(gb_v1)
        loss_.append(gb_loss_first.data)
        
        gb_first_train_op.zero_grad()
        gb_loss_first.backward()
        gb_first_train_op.step()

    output_from_first.append(gb_v1.tolist())
    print("1ST GBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, np.mean(loss_), mt))

output_from_first = torch.tensor(output_from_first)
for epoch in range(EPOCH):
    '''Second gbrbm'''
    for _, (data) in enumerate(output_from_first):
        try:
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue
        
        # CHANGED to GAUSSIAN
        sample_data = torch.normal(mean=data, std=gaussian_std)
        sample_data = torch.flatten(sample_data.clone())

        gb_vog_second, gb_v2, mt = rbm_second(sample_data)
        
        gb_loss_second = rbm_second.free_energy(gb_vog_second) - rbm_second.free_energy(gb_v2)
        loss_.append(gb_loss_second.data)
        
        gb_second_train_op.zero_grad()
        gb_loss_second.backward()
        gb_second_train_op.step()

    output_from_second.append(gb_v2.tolist())
    print("2ST GBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, np.mean(loss_), mt))

output_from_second = torch.tensor(output_from_second)
for epoch in range(EPOCH):
    '''Third gbrbm'''
    for _, (data) in enumerate(output_from_second):
        try:
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue
        
        # CHANGED to GAUSSIAN
        sample_data = torch.normal(mean=data, std=gaussian_std)
        sample_data = torch.flatten(sample_data.clone())

        gb_vog_third, gb_v3, mt = rbm_third(sample_data)
        
        gb_loss_third = rbm_third.free_energy(gb_vog_third) - rbm_second.free_energy(gb_v3)
        loss_.append(gb_loss_third.data)
        
        gb_third_train_op.zero_grad()
        gb_loss_third.backward()
        gb_third_train_op.step()

    print("3ST GBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, np.mean(loss_), mt))


BBRBM is done.
GBRBM is start


/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_38366/2809123372.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)


1ST GBrbm_first Training loss for 0 epoch -0.05427682027220726	Estimate time : 0:00:00.000104
1ST GBrbm_first Training loss for 1 epoch -0.054258741438388824	Estimate time : 0:00:00.000107
1ST GBrbm_first Training loss for 2 epoch -0.05424043908715248	Estimate time : 0:00:00.000148
1ST GBrbm_first Training loss for 3 epoch -0.05422261357307434	Estimate time : 0:00:00.000131
1ST GBrbm_first Training loss for 4 epoch -0.05420449376106262	Estimate time : 0:00:00.000111
1ST GBrbm_first Training loss for 5 epoch -0.05418660491704941	Estimate time : 0:00:00.000104
1ST GBrbm_first Training loss for 6 epoch -0.05416982248425484	Estimate time : 0:00:00.000105
1ST GBrbm_first Training loss for 7 epoch -0.05415158346295357	Estimate time : 0:00:00.000175
1ST GBrbm_first Training loss for 8 epoch -0.05413278192281723	Estimate time : 0:00:00.000105
1ST GBrbm_first Training loss for 9 epoch -0.05411499738693237	Estimate time : 0:00:00.000106
1ST GBrbm_first Training loss for 10 epoch -0.0540972873568

/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_38366/2809123372.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)


2ST GBrbm_first Training loss for 0 epoch -0.052503086626529694	Estimate time : 0:00:00.000104
2ST GBrbm_first Training loss for 1 epoch -0.05245883762836456	Estimate time : 0:00:00.000107
2ST GBrbm_first Training loss for 2 epoch -0.05241842195391655	Estimate time : 0:00:00.000105
2ST GBrbm_first Training loss for 3 epoch -0.05237594246864319	Estimate time : 0:00:00.000106
2ST GBrbm_first Training loss for 4 epoch -0.05233349651098251	Estimate time : 0:00:00.000111
2ST GBrbm_first Training loss for 5 epoch -0.052294738590717316	Estimate time : 0:00:00.000105
2ST GBrbm_first Training loss for 6 epoch -0.05225599557161331	Estimate time : 0:00:00.000106
2ST GBrbm_first Training loss for 7 epoch -0.0522138737142086	Estimate time : 0:00:00.000111
2ST GBrbm_first Training loss for 8 epoch -0.05216869339346886	Estimate time : 0:00:00.000105
2ST GBrbm_first Training loss for 9 epoch -0.05212518572807312	Estimate time : 0:00:00.000104
2ST GBrbm_first Training loss for 10 epoch -0.0520861335098

/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_38366/2809123372.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)


3ST GBrbm_first Training loss for 0 epoch -0.048462964594364166	Estimate time : 0:00:00.000112
3ST GBrbm_first Training loss for 1 epoch -0.048426225781440735	Estimate time : 0:00:00.000105
3ST GBrbm_first Training loss for 2 epoch -0.04838262498378754	Estimate time : 0:00:00.000106
3ST GBrbm_first Training loss for 3 epoch -0.04834313690662384	Estimate time : 0:00:00.000105
3ST GBrbm_first Training loss for 4 epoch -0.048306602984666824	Estimate time : 0:00:00.000117
3ST GBrbm_first Training loss for 5 epoch -0.0482657365500927	Estimate time : 0:00:00.000105
3ST GBrbm_first Training loss for 6 epoch -0.048231225460767746	Estimate time : 0:00:00.000106
3ST GBrbm_first Training loss for 7 epoch -0.04819716513156891	Estimate time : 0:00:00.000106
3ST GBrbm_first Training loss for 8 epoch -0.048159755766391754	Estimate time : 0:00:00.000106
3ST GBrbm_first Training loss for 9 epoch -0.04812271520495415	Estimate time : 0:00:00.000106
3ST GBrbm_first Training loss for 10 epoch -0.0480820238

In [13]:
nprst = gb_v3.detach().numpy()
print(nprst)

rbm_first.get_weight()

[1. 1. 1. 1. 1. 1. 0. 1. 1. 1.]


Parameter containing:
tensor([[-0.0073,  0.0054,  0.0027, -0.0168,  0.0127, -0.0131,  0.0142, -0.0156,
          0.0078, -0.0060]], requires_grad=True)

In [19]:
test_loss = 0
train_loss = 0
train_cnt = 0
summary_c = 0

for _, test_data in enumerate(test_dataloader):
    try:
        test_data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        train_data = torch.tensor((Variable(train_data[train_cnt:train_cnt + 10])).uniform_(0, 1), dtype=torch.float32)
    except RuntimeError:
        pass
    
    testing_data = torch.flatten(torch.bernoulli(test_data))
    training_data = torch.flatten(torch.bernoulli(train_data))
    
    vt, vt1, _ = rbm_first(testing_data)
    test_loss = rbm_first.free_energy(vt) - rbm_first.free_energy(vt1)    
    
    vs, vs1, _ = rbm_first(training_data)
    train_loss = rbm_first.free_energy(vs) - rbm_first.free_energy(vs1)

    
    test_loss += torch.mean(torch.abs(vt1[vt1 >= 0] - vt[vt1 >= 0]))
    print(vt1[vt1 >= 0] - vt[vt1 >= 0])
    summary_c += 1

print('Test loss : ' + str(test_loss / summary_c))
print('Train - test : ' + str(train_loss - test_loss))


/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_38366/285124220.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_38366/285124220.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_data = torch.tensor((Variable(train_data[train_cnt:train_cnt + 10])).uniform_(0, 1), dtype=torch.float32)
/Users/bahk_insung/miniforge3/lib/python3.9/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigm

tensor([ 1.,  0.,  0.,  0.,  1., -1., -1.,  0.,  0.,  1.],
       grad_fn=<SubBackward0>)
tensor([-1., -1., -1.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
       grad_fn=<SubBackward0>)
tensor([ 1.,  0.,  0., -1.,  1.,  0., -1.,  1.,  1., -1.],
       grad_fn=<SubBackward0>)
tensor([-1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  1.],
       grad_fn=<SubBackward0>)
tensor([0., 0., 0., 1., 0., 0., 1., 1., 1., 0.], grad_fn=<SubBackward0>)
tensor([ 0.,  0.,  1., -1.,  0.,  0., -1.,  1., -1., -1.],
       grad_fn=<SubBackward0>)
tensor([ 1.,  1.,  0.,  0., -1.,  0., -1.,  0.,  1., -1.],
       grad_fn=<SubBackward0>)
tensor([ 1.,  0.,  0., -1.,  0., -1.,  1.,  0.,  1.,  0.],
       grad_fn=<SubBackward0>)
tensor([1., 0., 0., 1., 0., 0., 0., 0., 0., 0.], grad_fn=<SubBackward0>)
tensor([ 1.,  0.,  1., -1.,  0.,  1.,  1.,  0.,  1.,  0.],
       grad_fn=<SubBackward0>)
tensor([ 1.,  0.,  1., -1.,  0., -1.,  0.,  0.,  0.,  0.],
       grad_fn=<SubBackward0>)
tensor([1., 1., 0., 0., 1., 0., 0., 0., 0., 